In [27]:
import requests
import pandas as pd
import sys
from io import BytesIO
import gzip
import io
import os
from global_variables import *
import sqlite3
from urllib.parse import urlparse

In [28]:
def extract_main_domain(url):
    #parsed_url = urlparse(url)
    # Split the hostname into subdomains and domain
    try:
        subdomains = str(url).split('.')
        if len(subdomains) > 2:
            # If there are subdomains, keep the last two parts
            return '.'.join(subdomains[-2:])
        else:
            # If no subdomains, return the whole hostname
            return url.lower()
    except:
        return url.lower()

In [29]:
urls = pd.read_csv('../data/FFIEC CDR Call Schedule RCM 12312021.txt',sep="\t")
urls = urls[['IDRSSD', 'TEXT4087']]
# urls.loc[len(urls)] = {'IDRSSD': '99999999', 'TEXT4087': 'djjon.com'}
# urls.loc[len(urls)] = {'IDRSSD': '999999991', 'TEXT4087': 'bocabrowandbeautybar.com'}
urls['url_cleaned'] = urls['TEXT4087'].str.lower()
urls.dropna(subset=['url_cleaned'], inplace=True)
urls.dropna(subset=['IDRSSD'], inplace=True)
urls['url_cleaned'] = urls['url_cleaned'].apply(extract_main_domain)
# urls.to_sql('bank_urls', conn, index=False, if_exists='replace')


In [30]:
results = requests.get(url=Monthly_Website_Traffic_API_PATH,
                           params={'page': 1},
                           headers={'X-API-KEY': API_KEY,
                                    'accept': 'application/json'
                                   })
response_json = results.json()

In [31]:
data_path = 'C:/Users/dratnadiwakara2/Downloads/temp_dewey/website_visits_banks.csv'
for link_data in response_json['download_links']:
        print(f"Downloading file {link_data['file_name']}...")
    
        data = requests.get(link_data['link'])
        gzip_stream = io.BytesIO(data.content)
    
        df = pd.read_csv(gzip_stream, compression='gzip', low_memory=False)
        df.dropna(subset=['DOMAIN'], inplace=True)
        df['DOMAIN'] = df['DOMAIN'].str.lower()
        df['DOMAIN_cleaned'] = df['DOMAIN'].apply(extract_main_domain)
        filtered_df = df[df['DOMAIN_cleaned'].isin(urls['url_cleaned'])]
    
        if os.path.isfile(data_path):
            header_option = False  
        else:
            header_option = True  
    
        filtered_df.to_csv(data_path, mode='a', header=header_option, index=False)        

In [32]:
ft_data = pd.read_csv(data_path,low_memory=False)

In [34]:
ft_data = ft_data.reset_index()
with gzip.open(data_path+'.gz', 'wt', encoding='utf-8') as gzipped_file:
    ft_data.to_csv(gzipped_file, index=False)